In [8]:
import pandas as pd
import time

# -------------------- Route Classes --------------------

from abc import ABC, abstractmethod

class Route(ABC):  # Abstract Base Class
    def __init__(self, route_id, distance, duration, cost, expression, variables):
        self.route_id = route_id
        self.distance = distance
        self.duration = duration
        self.cost = cost
        self.expression = expression
        self.variables = variables

    @abstractmethod
    def evaluate_expression(self):
        """Abstract method that must be implemented by subclasses."""
        pass

class ScenicRoute(Route):
    def __init__(self, route_id, distance, duration, cost, expression, variables, has_scenic_view=True):
        super().__init__(route_id, distance, duration, cost, expression, variables)
        self.has_scenic_view = has_scenic_view

    def evaluate_expression(self):
        try:
            return eval(self.expression, {}, self.variables)
        except Exception as e:
            print(f"Error in {self.route_id}: {e}")
            return False


# -------------------- Route Manager --------------------

class RouteManager:
    def __init__(self):
        self.routes = []

    def add_route(self, route):
        self.routes.append(route)

    def sort_by_cost_then_logic(self):
        for i in range(1, len(self.routes)):
            key = self.routes[i]
            j = i - 1
            while j >= 0 and (
                self.routes[j].cost > key.cost or
                (self.routes[j].cost == key.cost and not self.routes[j].evaluate_expression())
            ):
                self.routes[j + 1] = self.routes[j]
                j -= 1
            self.routes[j + 1] = key

    def merge_sort_by_duration_then_logic(self, routes=None):
        if routes is None:
            routes = self.routes
        if len(routes) <= 1:
            return routes
        mid = len(routes) // 2
        left = self.merge_sort_by_duration_then_logic(routes[:mid])
        right = self.merge_sort_by_duration_then_logic(routes[mid:])
        return self._merge(left, right)

    def _merge(self, left, right):
        result = []
        while left and right:
            if (left[0].duration < right[0].duration or
               (left[0].duration == right[0].duration and left[0].evaluate_expression())):
                result.append(left.pop(0))
            else:
                result.append(right.pop(0))
        result.extend(left or right)
        return result

    def search_routes_by_logic(self, logic_result=True):
        return [route for route in self.routes if route.evaluate_expression() == logic_result]

    def print_routes(self, routes=None):
        routes = routes or self.routes
        for r in routes:
            print(f"{r.route_id}: Duration={r.duration}, Cost={r.cost}, Eval={r.evaluate_expression()}")

# -------------------- Performance Analyzer --------------------

class PerformanceAnalyzer:
    @staticmethod
    def analyze(route_manager, method="insertion"):
        start = time.time()
        if method == "insertion":
            route_manager.sort_by_cost_then_logic()
        elif method == "merge":
            sorted_routes = route_manager.merge_sort_by_duration_then_logic()
            route_manager.routes = sorted_routes
        end = time.time()
        print(f"{method.title()} Sort Time: {end - start:.6f}s")

# -------------------- Load Data --------------------

def load_routes_from_csv(filename):
    df = pd.read_csv(filename)
    manager = RouteManager()
    for _, row in df.iterrows():
        route = ScenicRoute(
            route_id=row['route_id'],
            distance=float(row['distance']),
            duration=int(row['duration']),
            cost=float(row['cost']),
            expression=row['expression'],
            variables=eval(row['variables'])  # Safe only in controlled context
        )
        manager.add_route(route)
    return manager

# -------------------- Main Menu --------------------

def main():
    filename = "routes.csv"
    manager = load_routes_from_csv(filename)

    while True:
        print("\n===== Travel Planner Menu =====")
        print("1. Show all routes")
        print("2. Sort by cost (Insertion Sort) + logic")
        print("3. Sort by duration (Merge Sort) + logic")
        print("4. Search: Routes where logic is TRUE")
        print("5. Search: Routes where logic is FALSE")
        print("6. Exit")

        choice = input("Choose an option (1-6): ").strip()

        if choice == "1":
            manager.print_routes()
        elif choice == "2":
            PerformanceAnalyzer.analyze(manager, "insertion")
            manager.print_routes()
        elif choice == "3":
            PerformanceAnalyzer.analyze(manager, "merge")
            manager.print_routes()
        elif choice == "4":
            results = manager.search_routes_by_logic(True)
            manager.print_routes(results)
        elif choice == "5":
            results = manager.search_routes_by_logic(False)
            manager.print_routes(results)
        elif choice == "6":
            print("Exiting... 🚀")
            break
        else:
            print("Invalid option. Please choose again.")

# -------------------- Run --------------------

if __name__ == "__main__":
    main()




===== Travel Planner Menu =====
1. Show all routes
2. Sort by cost (Insertion Sort) + logic
3. Sort by duration (Merge Sort) + logic
4. Search: Routes where logic is TRUE
5. Search: Routes where logic is FALSE
6. Exit
R1: Duration=90, Cost=25.5, Eval=True
R2: Duration=60, Cost=18.0, Eval=True
R3: Duration=50, Cost=20.0, Eval=True
R4: Duration=110, Cost=30.0, Eval=True

===== Travel Planner Menu =====
1. Show all routes
2. Sort by cost (Insertion Sort) + logic
3. Sort by duration (Merge Sort) + logic
4. Search: Routes where logic is TRUE
5. Search: Routes where logic is FALSE
6. Exit
R1: Duration=90, Cost=25.5, Eval=True
R2: Duration=60, Cost=18.0, Eval=True
R3: Duration=50, Cost=20.0, Eval=True
R4: Duration=110, Cost=30.0, Eval=True

===== Travel Planner Menu =====
1. Show all routes
2. Sort by cost (Insertion Sort) + logic
3. Sort by duration (Merge Sort) + logic
4. Search: Routes where logic is TRUE
5. Search: Routes where logic is FALSE
6. Exit
Exiting... 🚀


In [9]:
!pip install matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 1.2 MB/s eta 0:00:0000:0100:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.2 MB/s eta 0:00:00a 0:00:01
